In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

In [2]:
#Parameters
file = "../tables/2021AP.xlsx"
next_week = 8
#Read in df
df = pd.read_excel(file)
df

Team  Week  Rank  W  L  Winning Percentage  Opp. Rank  \
0            Alabama     1     1  0  0                   1         14   
1           Oklahoma     1     2  0  0                   1         26   
2            Clemson     1     3  0  0                   1          5   
3         Ohio State     1     4  0  0                   1         26   
4            Georgia     1     5  0  0                   1          3   
..               ...   ...   ... .. ..                 ...        ...   
196  San Diego State     8    22  6  0                   1        NaN   
197       Pittsburgh     8    23  5  1            0.833333        NaN   
198             UTSA     8    24  7  0                   1        NaN   
199           Purdue     8    25  4  2            0.666667        NaN   
200             Team  Week  Rank  W  L  Winning Percentage  Opp. Rank   

     Opp. P5  Home  Result  Points Scored  Points Against  Margin  \
0          1     0       W             44              13      31   
1          0     1       W             40              35       5   
2          1     0       L              3              10      -7   
3          1     0       W             45              31      14   
4          1     0       W             10               3       7   
..       ...   ...     ...            ...             ...     ...   
196      NaN   NaN     NaN            NaN             NaN     NaN   
197      NaN   NaN     NaN            NaN             NaN     NaN   
198      NaN   NaN     NaN            NaN             NaN     NaN   
199      NaN   NaN     NaN            NaN             NaN     NaN   
200  Opp. P5  Home  Result  Points Scored  Points Against  Margin   

     Next Week Rank  Movement  
0                 1         0  
1                 4        -2  
2                 6        -3  
3                 3         1  
4                 2         3  
..              ...       ...  
196             NaN       NaN  
197             NaN       NaN  
198             NaN       NaN  
199             NaN       NaN  
200  Next Week Rank  Movement  

[201 rows x 15 columns]

In [3]:
#Drop footer
df.drop(index = (df.index.stop - 1), inplace = True)

#Drop overflow
for index, row in df.iterrows():
    if row['Week'] >= next_week:
        df.drop(index = index, inplace = True)

In [4]:
#Encode Result column
for index, row in df.iterrows():
    if row['Result'] == 'W':
        df.at[index, 'Result'] = 1
    else: 
        df.at[index, 'Result'] = 0

In [5]:
#Get next week teams list
next_teams = df['Team'].loc[df['Week'] == next_week - 1]
next_rank_act = df['Next Week Rank'].loc[df['Week'] == next_week - 1]

#Get byes
byes_index = []
for index, row in df.iterrows():
    if row['Week'] == next_week - 1 and not(row['Opp. Rank'] >= 0):
        byes_index.append(index % 25)
        
byes_index

[5, 6, 7, 13, 14, 15, 22]

In [6]:
#Drop null rows (teams with byes)
df.dropna(inplace = True)

In [7]:
#Drop irrelevant cols
df = df.drop(columns = ['Movement', 'Team'])

In [8]:
#Convert cols to num
for name, values in df.iteritems():
    df[name] = pd.to_numeric(values)

In [9]:
df.dtypes

Week                    int64
Rank                    int64
W                       int64
L                       int64
Winning Percentage    float64
Opp. Rank               int64
Opp. P5                 int64
Home                    int64
Result                  int64
Points Scored           int64
Points Against          int64
Margin                  int64
Next Week Rank          int64
dtype: object

In [10]:
#Set training data to Week < next week
train_df = df.loc[df['Week'] < next_week - 1]
X_train = train_df.drop(columns = ['Next Week Rank'])
y_train = train_df['Next Week Rank']

In [11]:
#Set testing data to Week = next week
test_df = df.loc[df['Week'] == next_week - 1]
X_test = test_df.drop(columns = ['Next Week Rank'])
y_test = test_df['Next Week Rank']

In [12]:
#Multiple Linear Regression
lin_model = LinearRegression().fit(X_train, y_train)
lin_model_output = lin_model.predict(X_test)
lin_model.score(X_test, y_test)

0.9440788530683298

In [13]:
#Add back teams with byes to output
next_rank_pred = lin_model_output.tolist()
for item in byes_index:
    next_rank_pred.insert(item, 'bye')

In [14]:
model_output = lin_model_output.tolist()
model_output

for item in byes_index:
    model_output.insert(item + 1, item + 1)
    
model_output.sort()    

sorted_output = model_output

In [15]:
df2 = pd.DataFrame()
df2['Predicted Rank'] = sorted_output
df2['Predicted Ordinal Rank'] = range(1, 26)

In [16]:
results = pd.DataFrame()
results['Team'] = next_teams
results['Predicted Rank'] = next_rank_pred
results['Actual Rank'] = next_rank_act

results.reset_index(inplace = True, drop = True)

#Predict that teams with byes don't change rank
for index, row in results.iterrows():
    if row['Predicted Rank'] == 'bye':
        results.at[index, 'Predicted Rank'] = index + 1

results['Predicted Rank'] = pd.to_numeric(results['Predicted Rank'], errors = 'coerce')
results['Actual Rank'] = pd.to_numeric(results['Actual Rank'])
results['Previous Rank'] = range(1, 26)

merged_results = results.merge(df2, how = 'left', left_on = 'Predicted Rank', right_on = 'Predicted Rank')

In [17]:
merged_results = merged_results[['Team', 'Previous Rank', 'Predicted Rank', 'Predicted Ordinal Rank', 'Actual Rank']]

In [18]:
merged_results

Team  Previous Rank  Predicted Rank  Predicted Ordinal Rank  \
0            Georgia              1       -2.164852                       1   
1               Iowa              2       11.871656                      11   
2         Cincinnati              3        3.767881                       3   
3           Oklahoma              4        2.546975                       2   
4            Alabama              5        5.480205                       4   
5         Ohio State              6        6.000000                       5   
6         Penn State              7        7.000000                       6   
7           Michigan              8        8.000000                       7   
8             Oregon              9       11.070088                      10   
9     Michigan State             10        8.464125                       8   
10          Kentucky             11       13.484801                      12   
11    Oklahoma State             12       10.710616                       9   
12          Ole Miss             13       14.167523                      14   
13        Notre Dame             14       14.000000                      13   
14  Coastal Carolina             15       15.000000                      15   
15       Wake Forest             16       16.000000                      16   
16          Arkansas             17       27.465974                      23   
17     Arizona State             18       26.039695                      21   
18               BYU             19       26.714215                      22   
19           Florida             20       28.694640                      24   
20         Texas A&M             21       20.707834                      18   
21          NC State             22       20.258029                      17   
22               SMU             23       23.000000                      20   
23   San Diego State             24       21.366122                      19   
24             Texas             25       30.317232                      25   

    Actual Rank  
0             1  
1            11  
2             2  
3             3  
4             4  
5             5  
6             7  
7             6  
8            10  
9             9  
10           15  
11            8  
12           12  
13           13  
14           14  
15           16  
16           26  
17           26  
18           26  
19           26  
20           17  
21           18  
22           21  
23           22  
24           26

In [19]:
merged_results.to_csv(f'week{next_week}_predictions.csv', header = True, index = False)